In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
import shutil

import preprocess_filter
import spoketools


In [24]:
def opusid_to_rpjb_path(opus_id):
    rpjb_path = None

    temp_list = glob.glob(f"../data/2023_rpjb/good/*/{opus_id}*")
    if len(temp_list) == 1:
        rpjb_path = temp_list[0]
    elif len(temp_list) == 0:
        print("path doesn't exist")
    else:
        print("there were multiple!")

    return rpjb_path

In [25]:
def mk_dict_for_training_group(images_path, spoke_type):
    filenames = glob.glob(images_path+"*.png")
    df_dict = {}

    df_dict['filenames'] = [x.split('/')[-1] for x in filenames]
    df_dict['opus_id'] = [x.split('.')[0] for x in df_dict['filenames']]
    df_dict['spoke_type'] = [spoke_type for x in range(0, len(df_dict['filenames']))]
    df_dict['rpjb_filename'] = [opusid_to_rpjb_path(x).split("/")[-1] for x in df_dict['opus_id'] if type(opusid_to_rpjb_path(x)) == str]


    return df_dict



Goal - create a csv that contains the following information about each opus id for data used in training. 
columns - filename, opus_id, spoke_type, indexs_uesd_for_cropping

In [26]:
light_dict = mk_dict_for_training_group("../data/training/light/images/", 'light')
dark_dict = mk_dict_for_training_group("../data/training/dark/images/", 'dark')
blank_dict = mk_dict_for_training_group("../data/training/blank/images/", 'blank')

In [31]:
light_df = pd.DataFrame(light_dict)
dark_df = pd.DataFrame(dark_dict)
blank_df = pd.DataFrame(blank_dict)

In [34]:
all_files_df = pd.concat([light_df, dark_df, blank_df])
all_files_df.set_index('opus_id', inplace=True)
all_files_df.to_csv("../data/training/all_training_data/data_catalog.csv")

In [4]:
all_files_df = pd.read_csv("../data/training/all_training_data/data_catalog.csv")
all_files_df

,opus_id,filenames,spoke_type,rpjb_filename
0,W1630586871,W1630586871.png,light,W1630586871_1_CALIB.rpjb
1,W1630553241,W1630553241.png,light,W1630553241_1_CALIB.rpjb
2,W1630639551,W1630639551.png,light,W1630639551_1_CALIB.rpjb
3,W1766588199,W1766588199.png,light,W1766588199_1_CALIB.rpjb
4,W1630785910,W1630785910.png,light,W1630785910_1_CALIB.rpjb
...,...,...,...,...
165,W1771110732,W1771110732.png,blank,W1771110732_1_CALIB.rpjb
166,W1771306117,W1771306117.png,blank,W1771306117_1_CALIB.rpjb
167,W1771114032,W1771114032.png,blank,W1771114032_1_CALIB.rpjb
168,W1771295813,W1771295813.png,blank,W1771295813_1_CALIB.rpjb


In [43]:
x = 0
for rpjb_filename, opus_id in zip(all_files_df['rpjb_filename'], all_files_df['opus_id']):
    rpjb_filepath = glob.glob(f"../data/2023_rpjb/good/*/{rpjb_filename}")[0]


    filename, pixel_values, coords = preprocess_filter.apply_filters(rpjb_filepath)
    pixel_values = preprocess_filter.apply_lucy_median(pixel_values)
    pixel_values = spoketools.fft2lpf(pixel_values, 0, 3)
    pixel_values, coords = preprocess_filter.buffer_image(pixel_values, 736, 160, coords)

    np.save(f"../data/training/all_training_data/as_numpy_cropped_filtered/np_{opus_id}.npy", pixel_values)

    print(opus_id , " is saved as a numpy array!")
        

W1630586871  is saved as a numpy array!
W1630553241  is saved as a numpy array!
W1630639551  is saved as a numpy array!
W1766588199  is saved as a numpy array!
W1630785910  is saved as a numpy array!
W1766553831  is saved as a numpy array!
W1630773329  is saved as a numpy array!
W1768388445  is saved as a numpy array!
W1630829947  is saved as a numpy array!
W1630845123  is saved as a numpy array!
W1630504290  is saved as a numpy array!
W1630471810  is saved as a numpy array!
W1766475429  is saved as a numpy array!
W1630772782  is saved as a numpy array!
W1630500810  is saved as a numpy array!
W1766581755  is saved as a numpy array!
W1630656351  is saved as a numpy array!
W1630481670  is saved as a numpy array!
W1630729022  is saved as a numpy array!
W1630470070  is saved as a numpy array!
W1770861322  is saved as a numpy array!
W1630632951  is saved as a numpy array!
W1630836451  is saved as a numpy array!
W1766490465  is saved as a numpy array!
W1766057124  is saved as a numpy array!


In [5]:
new_path = '../data/training/all_training_data/as_rpjb/'
for rpjb_filename, opus_id in zip(all_files_df['rpjb_filename'], all_files_df['opus_id']):
    rpjb_filepath = glob.glob(f"../data/2023_rpjb/good/*/{rpjb_filename}")[0]
    

    shutil.copy(rpjb_filepath, new_path+rpjb_filename)